# Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
KEYWORDS = ['python', 'парсинг', 'разработчик', 'ученые']

res =  requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html.parser') 
links = soup.find_all('h2', class_='post__title')
# извлечение всех тегов для ссылок
link_list = list(map(lambda x: x.find('a').get('href'), links))
# извлечение ссылок
link_list

[]

In [3]:
def get_habr_news(link_list):
    habr_news = pd.DataFrame()
    for link in link_list:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
#         запрос к ссылке
        time.sleep(0.3)
#     задержка на всякий случай
        text = soup.find('div', class_ = 'post__body post__body_full').text.lower().strip()
#     получение текста поста
        for words in KEYWORDS:
            if words in text:
#                 поиск нужных слов
                date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'), dayfirst = True).date()
#     получение даты
                title = soup.find('span', class_ = 'post__title-text').text
#     получение загловка
                row = {'date': date, 'title': title, 'link': link, 'text': text}
                habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
#         склеивание в dataframe
    return habr_news
get_habr_news(link_list)

""


# Задание 2.
Обязательная часть:

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [4]:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
HEADERS = {
    
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'ru-RU,ru;q=0.9,pl-PL;q=0.8,pl;q=0.7,en-US;q=0.6,en;q=0.5',
            'Connection': 'keep-alive',
            'Content-Length': '31',
            'Content-Type': 'application/json;charset=UTF-8',
            'Host': 'identityprotection.avast.com',
            'Origin': 'https://www.avast.com',
            'Referer': 'https://www.avast.com/',
            'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
            'sec-ch-ua-mobile': '?0',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-site',
            'User-Agent': 'Mozilla/4.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
            'Vaar-Header-App-Build-Version': '1.0.0',
            'Vaar-Header-App-Product': 'hackcheck-web-avast',
            'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
            'Vaar-Version': '0'
}
EMAILS = {
    'emailAddresses': ['xxx@x.ru']
}
EMAIL = ['xxx@x.ru','yyy@y.com']
leaks_frame = pd.DataFrame()
leaks_frame_new = pd.DataFrame()
for i in range(len(EMAIL)):
    EMAILS['emailAddresses'] = [EMAIL[i]]
    print(EMAILS)
    r = requests.post(URL, headers=HEADERS, json=EMAILS)
    time.sleep(5)
    #print(r)
    respond = r.json()

    for x in respond['breaches'].keys():
        leaks_frame['дата утечки']= [respond['breaches'][x]['publishDate']] 
        leaks_frame['источник утечки']= [respond['breaches'][x]['site']] 
        leaks_frame['описание утечки']= [respond['breaches'][x]['description']]
        leaks_frame_new = pd.concat([leaks_frame_new,leaks_frame])
leaks_frame_new

{'emailAddresses': ['xxx@x.ru']}
{'emailAddresses': ['yyy@y.com']}


,дата утечки,источник утечки,описание утечки
0,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,2022-11-10T00:00:00Z,gamersoul.com,"In May 2020, data belonging to the Bahamian-ba..."
0,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
0,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
0,2019-06-13T00:00:00Z,canva.com,"In May 2019, graphic-design site Canva's datab..."
